In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
url ="http://www.amazon.in/s?i=electronics&rh=n%3A976419031&dc&fs=true&ds=v1%3ArkkW9mgESmz%2BbycOM3sXv%2BcDZ1a9L4i22dh4iS0g1Wg"
custom_headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'}

In [3]:
response = requests.get(url, headers=custom_headers)
response

<Response [200]>

In [4]:
sp1 = BeautifulSoup(response.content, "html.parser")
product_items_dupli = sp1.find_all('div', class_='sg-col-inner')

In [7]:
import pandas as pd

# create empty product List
products = []

for product in product_items_dupli:
    try:
        product_name = product.find('span', class_='a-size-base-plus a-color-base a-text-normal').text.strip()
        product_price= product.find('span', class_ ='a-price').find("span", class_ ="a-offscreen").text.strip()
        product_rating = product.find('span', class_='a-icon-alt').text.strip()
        #print(product_rating)
        ## Product reviews
        #product_reviews = product.find('span', class_='reviews').text.strip()
        product_image_url = product.find('img', class_='s-image')['src']
        #print(product_image_url)
        # Add the product data to the list
        products.append({
            'Name': product_name,
            'Price': product_price,
            #'Description': product_description,
            'Rating': product_rating,
            #'Reviews': product_reviews,
            'Image_URL': product_image_url
        })

    except AttributeError:
        continue

# Convert the list to a DataFrame for better readability
df = pd.DataFrame(products)

In [94]:
import requests
from bs4 import BeautifulSoup

url = "http://www.amazon.in/s?i=electronics&rh=n%3A976419031&dc&fs=true&ds=v1%3ArkkW9mgESmz%2BbycOM3sXv%2BcDZ1a9L4i22dh4iS0g1Wg"

# Custom headers for bypass scraping restrictions
custom_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}

# Function to extract product data from the current page
def extract_product_data(soup):
    product_items = soup.find_all('div', class_='sg-col-inner')
    products = []
    for product in product_items:
        try:
            product_name = product.find('span', class_='a-size-base-plus a-color-base a-text-normal').text.strip()
            product_price = product.find('span', class_='a-price').find("span", class_="a-offscreen").text.strip()
            product_rating = product.find('span', class_='a-icon-alt').text.strip()
            product_image_url = product.find('img', class_='s-image')['src']

            # Add the product data to the list
            products.append({
                'Name': product_name,
                'Price': product_price,
                'Rating': product_rating,
                'Image_URL': product_image_url
            })
        except AttributeError:
            # Handle missing elements and continue to the next product
            continue
    return products

# Initial request to fetch the first page
response = requests.get(url, headers=custom_headers)
soup = BeautifulSoup(response.content, "html.parser")

# Initialize product data list
all_products = []

# Scrape all pages
while True:
    # Extract product data from the current page
    products = extract_product_data(soup)
    all_products.extend(products)

    # Find the next page URL (if available)
    next_page = soup.find('a', class_='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator')
    
    if next_page and 'href' in next_page.attrs:
        next_page_url = "https://www.amazon.in" + next_page['href']

        response1 = requests.get(next_page_url, headers=custom_headers)
        soup = BeautifulSoup(response1.content, 'html.parser')
    else:
    
        break

# Print or save the extracted product data
for product in all_products:
    print(product)


In [121]:
import requests
from bs4 import BeautifulSoup
import json

# Assign the website url
url = "http://www.amazon.com/s?i=electronics&rh=n%3A976419031&dc&fs=true"

# This part used to bypass scraping restrictions
custom_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}

# extract product data from the current page
def extract_product_data(soup):
    product_items = soup.find_all('div', class_='sg-col-inner')
    products = []
    for product in product_items:
        try:
            product_name = product.find('span', class_='a-size-base-plus a-color-base a-text-normal').text.strip()
            product_price = product.find('span', class_='a-price').find("span", class_="a-offscreen").text.strip()
            product_rating = product.find('span', class_='a-icon-alt').text.strip()
            product_image_url = product.find('img', class_='s-image')['src']

            # Add the product data to the list
            products.append({
                'Name': product_name,
                'Price': product_price,
                'Rating': product_rating,
                'Image_URL': product_image_url
            })
        except AttributeError:
            # Handle missing elements and continue to the next product
            continue
    return products

def scrape_multiple_pages(url, pages):
    all_products = []
    
    for page in range(1, pages + 1):
        # add the page number to the URL
        page_url = f"{url}&page={page}"
        
        # get the page content using requests
        response = requests.get(page_url, headers=custom_headers)
        
        # Request check area
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Extract product data from the current page
            products = extract_product_data(soup) 
            if products:
                all_products.extend(products)
        else:
            print(f"Failed to retrieve page {page}, status code: {response.status_code}")
    
    return all_products

# Scrape first 5 pages part
all_scraped_data = scrape_multiple_pages(url, 5)

# Final scraped data part
for product in all_scraped_data:
    print(product)
# json converted part    
with open('amazon_electronics.json', 'w') as file:
    json.dump(all_scraped_data, file, indent=4)

{'Name': 'I-4212E Mark II Printhead Replacement - PHD20-2278-01, 203 DPI, High-Performance Compatible Thermal Print Head for Datamax I-Class Mark II Barcode Label Printers', 'Price': '$169.00', 'Rating': '5.0 out of 5 stars', 'Image_URL': 'https://m.media-amazon.com/images/I/61gKuoWckvL._AC_UL320_.jpg'}
{'Name': 'RCA 10.1” WiFi Digital Picture Frame, Electronic Photo Frames 32GB Storage, HD IPS Touch Screen WiFi Frame/Auto Rotate/Wall Mounted, Easy Setup in Uhale App to Share Photo/Video Anytime (Black)', 'Price': '$109.99', 'Rating': '5.0 out of 5 stars', 'Image_URL': 'https://m.media-amazon.com/images/I/81Gq257yU7L._AC_UL320_.jpg'}
{'Name': 'Telescope, Telescopes for Adults Astronomy Professional, 90mm Aperture 1000mm High Powered Telescope for Beginners & Kids with Precision Adjustment Vertisteel AZ Mount Tripod Phone Adapter', 'Price': '$189.99', 'Rating': '2.5 out of 5 stars', 'Image_URL': 'https://m.media-amazon.com/images/I/61ERAtfPxZL._AC_UL320_.jpg'}
{'Name': 'Amazon Fire TV S

In [127]:
for product in all_scraped_data:
    print(product)
dt = pd.DataFrame(all_scraped_data)

{'Name': 'I-4212E Mark II Printhead Replacement - PHD20-2278-01, 203 DPI, High-Performance Compatible Thermal Print Head for Datamax I-Class Mark II Barcode Label Printers', 'Price': '$169.00', 'Rating': '5.0 out of 5 stars', 'Image_URL': 'https://m.media-amazon.com/images/I/61gKuoWckvL._AC_UL320_.jpg'}
{'Name': 'RCA 10.1” WiFi Digital Picture Frame, Electronic Photo Frames 32GB Storage, HD IPS Touch Screen WiFi Frame/Auto Rotate/Wall Mounted, Easy Setup in Uhale App to Share Photo/Video Anytime (Black)', 'Price': '$109.99', 'Rating': '5.0 out of 5 stars', 'Image_URL': 'https://m.media-amazon.com/images/I/81Gq257yU7L._AC_UL320_.jpg'}
{'Name': 'Telescope, Telescopes for Adults Astronomy Professional, 90mm Aperture 1000mm High Powered Telescope for Beginners & Kids with Precision Adjustment Vertisteel AZ Mount Tripod Phone Adapter', 'Price': '$189.99', 'Rating': '2.5 out of 5 stars', 'Image_URL': 'https://m.media-amazon.com/images/I/61ERAtfPxZL._AC_UL320_.jpg'}
{'Name': 'Amazon Fire TV S

In [128]:
dt

,Name,Price,Rating,Image_URL
0,I-4212E Mark II Printhead Replacement - PHD20-...,$169.00,5.0 out of 5 stars,https://m.media-amazon.com/images/I/61gKuoWckv...
1,"RCA 10.1” WiFi Digital Picture Frame, Electron...",$109.99,5.0 out of 5 stars,https://m.media-amazon.com/images/I/81Gq257yU7...
2,"Telescope, Telescopes for Adults Astronomy Pro...",$189.99,2.5 out of 5 stars,https://m.media-amazon.com/images/I/61ERAtfPxZ...
3,"Amazon Fire TV Stick, HD, sharp picture qualit...",$39.99,4.7 out of 5 stars,https://m.media-amazon.com/images/I/51TjJOTfsl...
4,Amazon Basics AAA Alkaline High-Performance Ba...,$13.67,4.7 out of 5 stars,https://m.media-amazon.com/images/I/81Apg8B6+0...
...,...,...,...,...
106,Amazon Basics 12-Pack C Cell Alkaline All-Purp...,$9.83,4.7 out of 5 stars,https://m.media-amazon.com/images/I/71QehdBsgk...
107,SAMSUNG Galaxy Buds 3 Pro AI True Wireless Blu...,$229.99,4.1 out of 5 stars,https://m.media-amazon.com/images/I/61zW8yc4hT...
108,"HP 952 Cyan, Magenta, Yellow Ink Cartridges (3...",$79.89,4.7 out of 5 stars,https://m.media-amazon.com/images/I/71tZw0rUWT...
109,ZHIYUN FIVERAY M20C Combo [Official] RGB Video...,$159.00,4.5 out of 5 stars,https://m.media-amazon.com/images/I/71vBxjH04S...
